In [7]:
pip install econml


Note: you may need to restart the kernel to use updated packages.


## Import Packages

In [8]:
from multiprocessing import Pool 

In [9]:
import pandas as pd 

In [10]:
from econml.dml import DML


In [11]:
from econml.iv.dml import DMLIV, NonParamDMLIV

In [12]:
from econml.iv.dml import OrthoIV

In [13]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier


In [6]:
from joblib import Parallel, delayed
import time as time_module

In [45]:
import numpy as np

## Importing Data

In [14]:
data  = pd.read_stata("workfile_china_expanded.dta")


In [35]:
len(data)

2311

In [15]:
reg_variables = [col for col in data.columns if col.startswith('reg')]

selected_columns = ["d_sh_empl_mfg","d_avg_lnwkwage", "d_tradeotch_pw_lag", "d_tradeusch_pw",
                    "l_shind_manuf_cbp", "l_sh_popedu_c", "l_sh_popfborn",
                    "l_sh_empl_f", "l_sh_routine33", "l_task_outsource", "timepwt48"] + reg_variables + ['t2', "statefip"]

data = data[selected_columns]

In [16]:
y = data["d_sh_empl_mfg"]
t = data["d_tradeusch_pw"]
z = data["d_tradeotch_pw_lag"]

In [21]:
ys = "d_sh_empl_mfg"
ts = "d_tradeusch_pw"
zs = "d_tradeotch_pw_lag"

cluster = "statefip"
ex_cols = ["statefip", "timepwt48"]

# Create conditions for excluding specific columns
exclude_conditions = [col_name not in [ys, ts, zs] + ex_cols for col_name in data.columns]

# Use the conditions to filter the column names
selected_columns = data.columns[exclude_conditions].tolist()

# Join the selected column names into a single string
w = data[selected_columns]

In [22]:
weight = data["timepwt48"]

 ## Defining ML Objects

In [36]:
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor

rf =  RandomForestRegressor(n_estimators=1000, min_samples_leaf=5)
glm = ElasticNetCV(cv=10, random_state=0)
gbm = GradientBoostingRegressor(min_samples_leaf = 1, learning_rate = 0.01)
regtree = DecisionTreeRegressor(random_state=0)
nn = MLPRegressor(random_state=0, max_iter=500)

### Ensemble


In [100]:
import mlens
import numpy as np

In [104]:

from mlens.ensemble import SuperLearner
from mlens.metrics import make_scorer
from sklearn.base import clone
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor



In [106]:
# Define the base learners
base_learners = [
    ('ml_l', glm),
    ('ml_b', gbm),
    ('ml_rt', rf),
    ('ml_rf', regtree)]

# Define the super learner
graph_ensemble_regr = SuperLearner(folds=5, scorer=make_scorer('mean_squared_error', greater_is_better=False))
graph_ensemble_regr.add(base_learners, proba=False)
graph_ensemble_regr.add_meta(LinearRegression())

# Fit the ensemble
graph_ensemble_regr.fit(y, t)  # Replace X_train and y_train with your data

# Convert the ensemble into a single learner
ml_nsmbl = clone(graph_ensemble_regr)


AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

## Parallel Processing

In [94]:
models = [rf, glm, gbm, regtree]
res_temp = []
import numpy as np

In [95]:
def fit_model(model):
    np.random.seed(1234)
    est = OrthoIV(model_y_xw = model,
           model_t_xw = model,
           model_t_xwz= model,
           projection= True,
           cv = 2,
           mc_iters = 100
         )
    est.fit(Y=y, T=t, Z=z, X=None, W=w, sample_weight= weight)
    summary = est.summary(alpha=0.05)
    res_temp.append(summary)
    return res_temp


In [96]:
results = Parallel(n_jobs=-1)(delayed(fit_model)(model) for model in models)

/Users/elkanfoudik/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/elkanfoudik/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/elkanfoudik/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/elkanfoudik/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected versi

Coefficient Results:  X is None, please call intercept_inference to learn the constant!
Coefficient Results:  X is None, please call intercept_inference to learn the constant!
Coefficient Results:  X is None, please call intercept_inference to learn the constant!
Coefficient Results:  X is None, please call intercept_inference to learn the constant!


In [98]:
results


[[<class 'econml.utilities.Summary'>
  """
                         CATE Intercept Results                       
                 point_estimate stderr zstat  pvalue ci_lower ci_upper
  --------------------------------------------------------------------
  cate_intercept         -0.337  0.052 -6.495    0.0   -0.439   -0.235
  --------------------------------------------------------------------
  
  <sub>A linear parametric conditional average treatment effect (CATE) model was fitted:
  $Y = \Theta(X)\cdot T + g(X, W) + \epsilon$
  where for every outcome $i$ and treatment $j$ the CATE $\Theta_{ij}(X)$ has the form:
  $\Theta_{ij}(X) = X' coef_{ij} + cate\_intercept_{ij}$
  Coefficient Results table portrays the $coef_{ij}$ parameter vector for each outcome $i$ and treatment $j$. Intercept Results table portrays the $cate\_intercept_{ij}$ parameter.</sub>
  """],
 [<class 'econml.utilities.Summary'>
  """
                         CATE Intercept Results                       
          